# Projet EMA

Ce projet est réalisé dans le cadre du cours Sécurité des composants à l'ENSTA Bretagne.

Source des données : (sur mon Google Drive pour l'instant) => penser à mettre le lien.

In [3]:
import matplotlib.pyplot as plt
import os
import re
import numpy as np
# import modin.pandas as pd
import pandas as pd

os.environ['MODIN_ENGINE'] = 'ray'  # Modin will use Ray

In [4]:
directory = os.path.join(os.getcwd(), 'TRACES_DATA')
tracesFiles = os.listdir(directory)
df = pd.read_csv(os.path.join(directory, tracesFiles[0]), sep=',', header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,0.003253,0.003226,0.003206,0.003196,0.003189,0.003189,0.003193,0.003203,0.003213,0.003223,...,0.002434,0.00249,0.002554,0.002627,0.002703,0.00279,0.002883,0.002986,0.003093,0.003209


## Etape 1 : Récupération des données brutes
On veut des matrices de la forme :

- keys : 20 000 lignes, 16 colonnes
- pti : 20 000 lignes, 16 colonnes
- cto : 20 000 lignes, 16 colonnes
- traces : 20 000 lignes, 4 000 colonnes


Ensuite, on les enregistre dans leur fichier respectif sous forme de matrices.

### Récupération des informations dans le nom des fichiers
On récupère les informations dans le nom des fichiers et on les stocke dans des matrices (`numpy`).
Les fichiers sont de la forme : `trace_AES_0000001_key=4c8cdf23b5c906f79057ec7184193a67_pti=8b5061cad6ae6aa4369eac99585d2bcb_cto=554b407d3634156c081abe31752ebd02.csv`

**Temps d'exécution : 1.9s**

In [15]:
keys = np.empty((len(tracesFiles), 16))
pti = np.empty((len(tracesFiles), 16))
cto = np.empty((len(tracesFiles), 16))
regex_filename = re.compile(
    r'\w*key=([a-z0-9]+)_pti=([a-z0-9]+)_cto=([a-z0-9]+).csv')
for i,filename in enumerate(tracesFiles):
    matchfilename = regex_filename.match(filename)
    if (matchfilename):
        keys_string = matchfilename.group(1)
        pti_string = matchfilename.group(2)
        cto_string = matchfilename.group(3)
        keys[i,:] = [int(keys_string[i:i+2], 16) for i in range(0, len(keys_string), 2)]
        pti[i,:] = [int(pti_string[i:i+2], 16) for i in range(0, len(pti_string), 2)]
        cto[i,:] = [int(cto_string[i:i+2], 16) for i in range(0, len(cto_string), 2)]
print(keys.shape, pti.shape, cto.shape)
print(keys[:5,:])


(20000, 16) (20000, 16) (20000, 16)
[[ 76. 140. 223.  35. 181. 201.   6. 247. 144.  87. 236. 113. 132.  25.
   58. 103.]
 [ 76. 140. 223.  35. 181. 201.   6. 247. 144.  87. 236. 113. 132.  25.
   58. 103.]
 [ 76. 140. 223.  35. 181. 201.   6. 247. 144.  87. 236. 113. 132.  25.
   58. 103.]
 [ 76. 140. 223.  35. 181. 201.   6. 247. 144.  87. 236. 113. 132.  25.
   58. 103.]
 [ 76. 140. 223.  35. 181. 201.   6. 247. 144.  87. 236. 113. 132.  25.
   58. 103.]]


### Récupération des traces
On récupère les traces dans les fichiers et on les stocke dans une matrice (`numpy`).
Elle doit être de taille : 
- traces : 20 000 lignes, 4 000 colonnes

**Temps d'exécution : 12s pour 1000 valeurs** \
**Temps estimé pour 20 000 valeurs : 4min** \
**Temps d'exécution : 4min 55s**

In [25]:
traces = np.empty((len(tracesFiles), 4000))

for i, filename in enumerate(tracesFiles):
    fullPath = os.path.join(directory, filename)
    traces[i,:] = np.fromfile(fullPath, dtype=float, sep=',')

print(traces.shape)
print(traces[:5])

(20000, 4000)
[[ 0.0032526   0.003226    0.003206   ...  0.0029863   0.0030928
   0.0032093 ]
 [ 0.0021939   0.0021973   0.0022039  ...  0.0027399   0.0027066
   0.0026634 ]
 [ 0.0022672   0.0022971   0.0023238  ...  0.00037953  0.00025302
   0.00012984]
 [-0.00082231 -0.00072243 -0.0006159  ...  0.00096879  0.00092551
   0.00087225]
 [-0.0016313  -0.0015914  -0.0015348  ... -0.00028631 -0.0003762
  -0.00046276]]


In [31]:
print(traces[-5:])
# df = pd.DataFrame()
# for filename in tracesFiles[:100]:
#     df = pd.read_csv(os.path.join(directory, filename), sep=',')
# df
for n in range(traces.shape[0]):
    plt.plot(traces[n,:])
plt.show()

[[ 0.007524    0.0073775   0.0072243  ...  0.0013283   0.0013084
   0.0012817 ]
 [ 0.00059925  0.0006725   0.00074241 ...  0.0033358   0.0035123
   0.0036721 ]
 [-0.0028465  -0.0029164  -0.0029663  ...  0.00096879  0.00086559
   0.00075573]
 [ 0.0010953   0.00098544  0.00088556 ...  0.0045177   0.0044445
   0.0043679 ]
 [ 0.0052035   0.0052901   0.0053567  ... -0.0015914  -0.0015281
  -0.0014715 ]]


KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x000002A17F551820> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000002A101E8F550> (for post_execute):


KeyboardInterrupt: 

In [32]:
# Initialisation de la matrice pour stocker les données
data_matrix = np.empty((0,4))
data_pandas = pd.DataFrame()

def readcsv_numpy(filename, data_matrix):
    data = np.genfromtxt(filename, delimiter=",")
    data_matrix = np.append(data_matrix.reshape(data_matrix.shape[0],data.shape[0]), data.reshape(1,data.shape[0]), axis=0)
    return data_matrix
def readcsv_pandas(filename:str, data_pandas: pd.DataFrame):
    data = pd.read_csv(filename, sep=',', header=None)
    data_pandas = pd.concat([data_pandas, data], axis=0)
    return data_pandas

# Boucle pour lire les fichiers CSV
for file in tracesFiles[:100]:
    data = pd.read_csv(filename, sep=',', header=None)
    data_pandas = pd.concat([data_pandas, data], axis=0)

# Afficher la matrice
data_pandas.head()

FileNotFoundError: [Errno 2] No such file or directory: 'trace_AES_0020000_key=4c8cdf23b5c906f79057ec7184193a67_pti=2b3a31a2abb2139580f7f4e42692882f_cto=bfdb67fe0c13d4893cf746406e762c69.csv'